In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
leaf_stop=15
df = pd.read_csv("cfs_formatted.csv")

In [ ]:
# HYP 1

# num = 41
# Identify separator rows and create a group for each original data section
df = pd.read_csv("cfs_formatted.csv")
separator_mask = df.apply(lambda row: all(cell == "---" for cell in row), axis=1)
df['group_id'] = separator_mask.cumsum()

# List to collect paired results
all_paired_rows = []

# Iterate over groups to find pairs within each
for group_id, group_df in df.groupby('group_id'):
    # Filter out separator rows within each group
    group_df = group_df[~separator_mask]

    # Identify rows excluding the original row for pairing
    proximity_rows = group_df[group_df['source'].str.startswith('prox')]

    # Check if there are at least two rows for pairing
    if len(proximity_rows) > 1:
        # Treat the first row as the 'original_data' row for context
        original_data_row = group_df[group_df['source'] == 'original'].iloc[0]

        # Temporary list to collect pairs for this group
        group_paired_rows = []

        # Iterate over all possible pairs of proximity rows
        for i in range(len(proximity_rows)):
            for j in range(len(proximity_rows)):
                if i != j:  # Ensure we are comparing different rows
                    row_i = proximity_rows.iloc[i]
                    row_j = proximity_rows.iloc[j]
                    if (
                        original_data_row['income'] <= row_i['income'] and 
                        original_data_row['income'] <= row_j['income'] and 
                        original_data_row['credit_score'] <= row_i['credit_score'] and 
                        original_data_row['credit_score'] <= row_j['credit_score'] and 
                        original_data_row['amount_requested'] >= row_i['amount_requested'] and 
                        original_data_row['amount_requested'] >= row_j['amount_requested']
                    ):
                        if original_data_row['employment_type'] == 'private':
                            condition = (row_i['employment_type'] in ('private', 'govt')) and (row_j['employment_type'] in ('private', 'govt'))
                        elif original_data_row['employment_type'] == 'govt':
                            condition = (row_i['employment_type'] == 'govt') and (row_j['employment_type'] == 'govt')
                        elif original_data_row['employment_type'] == 'self':
                            condition = (row_i['employment_type'] in ('self', 'private', 'govt')) and (row_j['employment_type'] in ('self', 'private', 'govt'))
                        else:
                            condition = False  # or handle other cases

                        if condition:
                            if original_data_row['education_type'] == 'less_than_high_school':
                                condition2 = (row_i['education_type'] in ("less_than_high_school", "highschool", "associate", "bachelors", "advanced")) and (row_j['education_type'] in ("less_than_high_school", "highschool", "associate", "bachelors", "advanced"))
                            elif original_data_row['education_type'] == 'highschool':
                                condition2 = (row_i['education_type'] in ("highschool", "associate", "bachelors", "advanced")) and (row_j['education_type'] in ("highschool", "associate", "bachelors", "advanced"))
                            elif original_data_row['education_type'] == 'associate':
                                condition2 = (row_i['education_type'] in ("associate", "bachelors", "advanced")) and (row_j['education_type'] in ("associate", "bachelors", "advanced"))
                            elif original_data_row['education_type'] == 'bachelors':
                                condition2 = (row_i['education_type'] in ("bachelors", "advanced")) and (row_j['education_type'] in ("bachelors", "advanced"))
                            elif original_data_row['education_type'] == 'advanced':
                                condition2 = (row_i['education_type'] == "advanced") and (row_j['education_type'] == "advanced")
                            else:
                                condition2 = False
                            if condition2:
                                # Ensure numeric conversion with error handling
                                row_i['true_proximity'] = float(row_i['true_proximity'])
                                row_j['true_proximity'] = float(row_j['true_proximity'])
                                row_i['weighted_proximity'] = float(row_i['weighted_proximity'])
                                row_j['weighted_proximity'] = float(row_j['weighted_proximity'])

                                # Check if the pair meets the mismatch condition
                                if (
                                    row_i['true_proximity'] > row_j['true_proximity'] and
                                    row_i['weighted_proximity'] < row_j['weighted_proximity']
                                ):
                                    # Blank out unchanged features for row_i
                                    greater_row = row_i.copy()
                                    for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                                        if greater_row[column] == original_data_row[column]:
                                            greater_row[column] = ""  # Blank out if unchanged

                                    # Blank out unchanged features for row_j
                                    lesser_row = row_j.copy()
                                    for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                                        if lesser_row[column] == original_data_row[column]:
                                            lesser_row[column] = ""  # Blank out if unchanged

                                    group_paired_rows.append(greater_row.to_dict())
                                    group_paired_rows.append(lesser_row.to_dict())
                                    # Append a separator row for readability
                                    separator = {col: "---" for col in df.columns}
                                    group_paired_rows.append(separator)
        
        # Only append the original row if there are pairs in this group
        if group_paired_rows:
            all_paired_rows.append(original_data_row.to_dict())
            all_paired_rows.extend(group_paired_rows)

# Output debug information if no pairs were found
if not all_paired_rows:
    print("No pairs found. Review the mismatch condition or the data structure.")
    
 
output_df = pd.DataFrame(all_paired_rows)
output_df.to_csv("hyp1_cfs.csv", index=False)   





In [ ]:
# HYP 2

round_df = pd.read_csv("round_cfs_formatted.csv")

# Identify separator rows and create a group for each original data section
separator_mask = round_df.apply(lambda row: all(cell == "---" for cell in row), axis=1)
round_df['group_id'] = separator_mask.cumsum()

# List to collect valid paired groups
all_paired_rows = []

# Iterate over groups to filter and format data as specified
for group_id, group_df in round_df.groupby('group_id'):
    # Filter out separator rows within each group
    group_df = group_df[~separator_mask]

    # Check if there's an original row in the group
    original_data_row = group_df[group_df['source'] == "original"]
    if original_data_row.empty:
        continue
    original_data_row = original_data_row.iloc[0]

    # Collect prox and round_prox pairs for the current group
    valid_prox_rows = []
    for prox_num in range(1, leaf_stop):
        prox_row = group_df[group_df['source'] == f"prox{prox_num}"]
        round_prox_row = group_df[group_df['source'] == f"round_prox{prox_num}"]

        # Include only complete prox and round_prox pairs
        if not prox_row.empty and not round_prox_row.empty:
            prox_row = prox_row.iloc[0]
            round_prox_row = round_prox_row.iloc[0]

            if (
                original_data_row['income'] <= prox_row['income'] and 
                original_data_row['credit_score'] <= prox_row['credit_score'] and 
                original_data_row['amount_requested'] >= prox_row['amount_requested']
            ):
                if original_data_row['employment_type'] == 'private':
                    condition = (prox_row['employment_type'] in ('private', 'govt'))
                elif original_data_row['employment_type'] == 'govt':
                    condition = (prox_row['employment_type'] == 'govt')
                elif original_data_row['employment_type'] == 'self':
                    condition = (prox_row['employment_type'] in ('self', 'private', 'govt'))
                else:
                    condition = False  # or handle other cases
                    
                if condition:
                    if original_data_row['education_type'] == 'less_than_high_school':
                        condition2 = (prox_row['education_type'] in ("less_than_high_school", "highschool", "associate", "bachelors", "advanced"))
                    elif original_data_row['education_type'] == 'highschool':
                        condition2 = (prox_row['education_type'] in ("highschool", "associate", "bachelors", "advanced"))
                    elif original_data_row['education_type'] == 'associate':
                        condition2 = (prox_row['education_type'] in ("associate", "bachelors", "advanced"))
                    elif original_data_row['education_type'] == 'bachelors':
                        condition2 = (prox_row['education_type'] in ("bachelors", "advanced"))
                    elif original_data_row['education_type'] == 'advanced':
                        condition2 = (prox_row['education_type'] == "advanced")
                    else:
                        condition2 = False
                        
                    if condition2:
                        # Blank unchanged features for prox_row
                        prox_row_modified = prox_row.copy()
                        for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                            if prox_row_modified[column] == original_data_row[column]:
                                prox_row_modified[column] = ""  # Blank out if unchanged
                        valid_prox_rows.append(prox_row_modified.to_dict())

                        # Blank unchanged features for round_prox_row
                        round_prox_row_modified = round_prox_row.copy()
                        for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                            if round_prox_row_modified[column] == original_data_row[column]:
                                round_prox_row_modified[column] = ""  # Blank out if unchanged
                        valid_prox_rows.append(round_prox_row_modified.to_dict())

    if valid_prox_rows:
        all_paired_rows.append(original_data_row.to_dict())
        all_paired_rows.extend(valid_prox_rows)

        # Append a separator row after each original group for readability
        separator = {col: "---" for col in df.columns}
        all_paired_rows.append(separator)

In [ ]:
# Convert the paired rows to a DataFrame and output to CSV
output_df = pd.DataFrame(all_paired_rows)
output_df.to_csv("hyp2_cfs.csv", index=False)

In [ ]:
# HYP 3

df = pd.read_csv("cfs_formatted.csv")

# Identify separator rows and create a group for each original data section
separator_mask = df.apply(lambda row: all(cell == "---" for cell in row), axis=1)
df['group_id'] = separator_mask.cumsum()

# List to collect paired results
all_paired_rows = []

# Iterate over groups to find pairs within each
for group_id, group_df in df.groupby('group_id'):
    # Filter out separator rows within each group
    group_df = group_df[~separator_mask]

    # Identify rows excluding the original row for pairing
    proximity_rows = group_df[group_df['source'].str.startswith('prox')]

    # Check if there are at least two rows for pairing
    if len(proximity_rows) > 1:
        # Treat the first row as the 'original_data' row for context
        original_data_row = group_df[group_df['source'] == 'original'].iloc[0]

        # Temporary list to collect pairs for this group
        group_paired_rows = []

        # Iterate over all possible pairs of proximity rows
        for i in range(len(proximity_rows)):
            for j in range(len(proximity_rows)):
                if i != j:  # Ensure we are comparing different rows
                    row_i = proximity_rows.iloc[i]
                    row_j = proximity_rows.iloc[j]

                    # Ensure numeric conversion with error handling
                    row_i['true_proximity'] = float(row_i['true_proximity'])
                    row_j['true_proximity'] = float(row_j['true_proximity'])
                        
                    # Blank out unchanged features for row_i
                    greater_row = row_i.copy()
                    for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                        if greater_row[column] == original_data_row[column]:
                            greater_row[column] = ""  # Blank out if unchanged

                    # Blank out unchanged features for row_j
                    lesser_row = row_j.copy()
                    for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                        if lesser_row[column] == original_data_row[column]:
                            lesser_row[column] = ""  # Blank out if unchanged
                    
                    # See if only categorical changes were made
                    if ((greater_row['employment_type']!="" or greater_row['education_type']!="") and
                        (greater_row['income']=="" and greater_row['credit_score']=="" and greater_row['amount_requested']=="") and
                        (lesser_row['employment_type']=="" and lesser_row['education_type']=="")):
                        i_cat_change = True
                    else:
                        i_cat_change = False

                    if i_cat_change:
                        group_paired_rows.append(greater_row.to_dict())
                        group_paired_rows.append(lesser_row.to_dict())
                        # Append a separator row for readability
                        separator = {col: "---" for col in df.columns}
                        group_paired_rows.append(separator)

        # Only append the original row if there are pairs in this group
        if group_paired_rows:
            all_paired_rows.append(original_data_row.to_dict())
            all_paired_rows.extend(group_paired_rows)

# Output debug information if no pairs were found
if not all_paired_rows:
    print("No pairs found. Review the mismatch condition or the data structure.")


In [ ]:
# Convert the paired rows to a DataFrame and output to CSV
output_df = pd.DataFrame(all_paired_rows)
output_df.to_csv("hyp3_cfs.csv", index=False)

In [ ]:
# HYP 4

df = pd.read_csv("cfs_formatted.csv")

# Identify separator rows and create a group for each original data section
separator_mask = df.apply(lambda row: all(cell == "---" for cell in row), axis=1)
df['group_id'] = separator_mask.cumsum()

# List to collect paired results
all_paired_rows = []

# Iterate over groups to find pairs within each
for group_id, group_df in df.groupby('group_id'):
    # Filter out separator rows within each group
    group_df = group_df[~separator_mask]

    # Identify rows excluding the original row for pairing
    proximity_rows = group_df[group_df['source'].str.startswith('prox')]

    # Check if there are at least two rows for pairing
    if len(proximity_rows) > 1:
        # Treat the first row as the 'original_data' row for context
        original_data_row = group_df[group_df['source'] == 'original'].iloc[0]

        # Temporary list to collect pairs for this group
        group_paired_rows = []

        # Iterate over all possible pairs of proximity rows
        for i in range(len(proximity_rows)):
            for j in range(len(proximity_rows)):
                if i != j:  # Ensure we are comparing different rows
                    row_i = proximity_rows.iloc[i]
                    row_j = proximity_rows.iloc[j]

                    # Ensure numeric conversion with error handling
                    row_i['true_proximity'] = float(row_i['true_proximity'])
                    row_j['true_proximity'] = float(row_j['true_proximity'])

                    # Check if j is closer than i
                    if (
                        row_i['true_proximity'] > row_j['true_proximity']
                    ):

                        greater_row = row_i.copy()
                        
                        if (float(original_data_row['credit_score']) > float(greater_row['credit_score']) or
                            float(original_data_row['income']) > float(greater_row['income']) or
                            float(original_data_row['amount_requested']) < float(greater_row['amount_requested'])):
                            i_counterint = True
                        else:
                            i_counterint = False

                        # Blank out unchanged features for row_i
                        for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                            if greater_row[column] == original_data_row[column]:
                                greater_row[column] = ""  # Blank out if unchanged

                        lesser_row = row_j.copy()

                        if (float(original_data_row['credit_score']) > float(lesser_row['credit_score']) or
                            float(original_data_row['income']) > float(lesser_row['income']) or
                            float(original_data_row['amount_requested']) < float(lesser_row['amount_requested'])):
                            j_counterint = True
                        else:
                            j_counterint = False
                        
                        # Blank out unchanged features for row_j
                        for column in ['income', 'credit_score', 'amount_requested', 'employment_type', 'education_type']:
                            if lesser_row[column] == original_data_row[column]:
                                lesser_row[column] = ""  # Blank out if unchanged

                        # Check if j is closer but counterintuitive
                        if j_counterint and not i_counterint:
                            group_paired_rows.append(greater_row.to_dict())
                            group_paired_rows.append(lesser_row.to_dict())
                            # Append a separator row for readability
                            separator = {col: "---" for col in df.columns}
                            group_paired_rows.append(separator)

        # Only append the original row if there are pairs in this group
        if group_paired_rows:
            all_paired_rows.append(original_data_row.to_dict())
            all_paired_rows.extend(group_paired_rows)

# Output debug information if no pairs were found
if not all_paired_rows:
    print("No pairs found. Review the mismatch condition or the data structure.")


In [ ]:
# Convert the paired rows to a DataFrame and output to CSV
output_df = pd.DataFrame(all_paired_rows)
output_df.to_csv("hyp4_cfs.csv", index=False)